**Keypoint Detection using Detectron2**

https://detectron2.readthedocs.io/en/latest/tutorials/models.html#model-output-format

**COCO keypoint detection Format**

**"name":** "person", # (specific category)

**"supercategory":** "person", #

**"id":** 1, # class id

**"keypoints":** [
"nose",
"left_eye",
"right_eye",
"left_ear",
"right_ear",
"left_shoulder",
"right_shoulder",
"left_elbow",
"right_elbow",
"left_wrist",
"right_wrist",
"left_hip",
"right_hip",
"left_knee",
"right_knee",
"left_ankle",
"right_ankle"]

**The detectron2 outputs 17 keypoints**

**Skeleton :** in pairs e.g [16,14] means ["left_ankle","left_knee"] (connection)

**Note :** for training your own  model , data should be on COCO format just like above .

**Import Libs**

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [11]:
# import some common libraries
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

**Testing Dataset**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Path to the directory containing your images in Google Drive
drive_image_dir ="/content/drive/MyDrive/imgs_test_set"

**Load Pretrained keypoint detection Model**

In [18]:
#Meta data : Understand what we have in hands
network_model="COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"
cls = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
print('NetworkModel[%s] category:%d'%(network_model, len(cls)))
print(cls)

kname = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).keypoint_names
print('keypoint_names')
print(kname)

krule = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).keypoint_connection_rules
print('keypoint_connection_rules')
print(krule)

NetworkModel[COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml] category:1
['person']
keypoint_names
('nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle')
keypoint_connection_rules
[('left_ear', 'left_eye', (102, 204, 255)), ('right_ear', 'right_eye', (51, 153, 255)), ('left_eye', 'nose', (102, 0, 204)), ('nose', 'right_eye', (51, 102, 255)), ('left_shoulder', 'right_shoulder', (255, 128, 0)), ('left_shoulder', 'left_elbow', (153, 255, 204)), ('right_shoulder', 'right_elbow', (128, 229, 255)), ('left_elbow', 'left_wrist', (153, 255, 153)), ('right_elbow', 'right_wrist', (102, 255, 224)), ('left_hip', 'right_hip', (255, 102, 0)), ('left_hip', 'left_knee', (255, 255, 77)), ('right_hip', 'right_knee', (153, 255, 204)), ('left_knee', 'left_ankle', (191, 255, 128)), ('right_knee', 'right_ankle', (255, 195, 77))]


In [8]:
# Create the Detectron2 configuration
cfg = get_cfg()
#change device
cfg.MODEL.DEVICE = "cpu"
# load the pre trained model from Detectron2 model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
# set confidence threshold for this model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
# load model weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
# create the predictor for pose estimation using the config
pose_detector = DefaultPredictor(cfg)

[11/06 10:20:30 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...


model_final_a6e10b.pkl: 237MB [00:02, 112MB/s]                           


**Save keypoints detection Results**

In [14]:
#output
output_dir="/content/drive/MyDrive/imgs_test_set/2D_pose/detctron2"

In [17]:
for filename in os.listdir(drive_image_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(drive_image_dir, filename)

        im = cv2.imread(image_path)

        # Run object detection on the image
        predictor = DefaultPredictor(cfg)
        outputs = predictor(im)

        # Create a visualizer and draw instance predictions
        v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        # Save the result image to the output directory
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, out.get_image()[:, :, ::-1])


[11/06 10:54:18 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...
[11/06 10:54:27 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...
[11/06 10:54:46 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...
[11/06 10:54:58 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...
[11/06 10:55:13 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn

For complete exploration , we will print the keypoints along with the names and the N persons detected on each image

In [21]:
for filename in os.listdir(drive_image_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(drive_image_dir, filename)

        im = cv2.imread(image_path)

        # Run object detection on the image
        predictor = DefaultPredictor(cfg)
        outputs = predictor(im)

        # Access the keypoints using get_fields method
        keypoints = outputs["instances"].get("pred_keypoints")

        if keypoints is not None:
            for instance_keypoints in keypoints:
                print("Keypoint Names:", MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).keypoint_names)
                print("Number of Detected Persons:", len(keypoints))
                print("Instance Keypoints:", instance_keypoints)

        # Create a visualizer and draw instance predictions
        v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        # Save the result image to the output directory
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

[11/06 11:16:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...
Keypoint Names: ('nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle')
Number of Detected Persons: 4
Instance Keypoints: tensor([[5.3201e+02, 5.2174e+02, 9.4101e-01],
        [5.4305e+02, 5.1714e+02, 9.5398e-01],
        [5.3753e+02, 5.1530e+02, 8.5509e-01],
        [5.6696e+02, 5.3460e+02, 8.9412e-01],
        [5.6604e+02, 5.3552e+02, 4.7996e-01],
        [5.6144e+02, 5.8237e+02, 1.8839e-01],
        [5.3937e+02, 5.6032e+02, 1.1752e-01],
        [4.8235e+02, 5.8053e+02, 4.8382e-01],
        [4.8511e+02, 5.5389e+02, 3.2208e-01],
        [4.1981e+02, 5.5113e+02, 4.1361e-01],
        [4.3085e+02, 5.4195e+02